In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from datetime import timedelta
from datetime import datetime
import statistics

## Code to create a matrix

In [6]:
def scan_params(data_file, total_build_factors, solar_build_fractions, num_evs, ev_capacity_factor, behavioral):
    hourly_data = pd.read_csv(data_file)
    ev_battery_size = 0.05 #MWh
    frac_by_hour = {}
    max_ev_storage = ev_capacity_factor*ev_battery_size*num_evs

    hourly_data = hourly_data.sort_values(by='time')

    total_demand = hourly_data['demand'].sum()
    total_solar = hourly_data['total_SWGDN'].sum()
    total_wind = hourly_data['total_wind_capacity'].sum()

    matrix = []
    for build_factor in total_build_factors:
        matrix_row = []
        for solar_frac in solar_build_fractions:
            total_energy_used = 0

            solar_build_factor = solar_frac*build_factor
            wind_build_factor = build_factor - solar_build_factor
            
            solar_adjustment = solar_build_factor * total_demand / total_solar
            wind_adjustment = wind_build_factor * total_demand / total_wind

            hourly_data['built_wind'] = hourly_data['total_wind_capacity'] * wind_adjustment
            hourly_data['built_solar'] = hourly_data['total_SWGDN'] * solar_adjustment

            energy_in_storage = max_ev_storage*0.5
            met_demand, unmet_demand = 0, 0
            for _, row in hourly_data.iterrows():
                total_supply = row['built_wind'] + row['built_solar']
                if total_supply > row['demand']:
                    if behavioral and ('08:30' in row['time'] or '17:30' in row['time']):
                        pass
                    else:
                        energy_in_storage = min(max_ev_storage, energy_in_storage + total_supply - row['demand'])
                    met_demand += row['demand']
                else:
                    gap = row['demand'] - total_supply
                    if behavioral and ('08:30' in row['time'] or '17:30' in row['time']):
                        energy_used = 0
                    elif gap > energy_in_storage:
                        energy_used = energy_in_storage
                    else:
                        energy_used = gap
                    total_energy_used += energy_used
                    energy_in_storage -= energy_used
                    met_demand += energy_used + total_supply
                    unmet_demand += row['demand'] - total_supply - energy_used
                if max_ev_storage != 0 and build_factor == 1 and solar_frac == 0.75:
                    frac_by_hour.setdefault(row['time'].split()[-1], []).append(energy_in_storage / max_ev_storage)
            total_met_demand = met_demand/total_demand
            matrix_row.append(float(total_met_demand))
            if num_evs != 0:
                charge_cycles = total_energy_used / num_evs / ev_battery_size
                # print(charge_cycles)
            # print(total_energy_used)
        matrix.append(matrix_row)
    return matrix, frac_by_hour

## Plot demand and supply

In [7]:
solar_build_factor = 1
wind_build_factor = 1
hourly_data = pd.read_csv('CA2021hourly.csv')

total_demand = hourly_data['demand'].sum()
total_solar = hourly_data['total_SWGDN'].sum()
total_wind = hourly_data['total_wind_capacity'].sum()
    
solar_adjustment = solar_build_factor * total_demand / total_solar
wind_adjustment = wind_build_factor * total_demand / total_wind

hourly_data['built_wind'] = hourly_data['total_wind_capacity'] * wind_adjustment
hourly_data['built_solar'] = hourly_data['total_SWGDN'] * solar_adjustment

hourly_data['Date'] = hourly_data['time']
hourly_data['Demand (MW)'] = hourly_data['demand']

print(sum(hourly_data['Demand (MW)']))
hourly_data['Wind Power Built (MW)'] = hourly_data['built_wind']
hourly_data['Solar Power Built (MW)'] = hourly_data['built_solar']
for var in['Demand (MW)', 'Wind Power Built (MW)', 'Solar Power Built (MW)']:
    fig = px.line(hourly_data, x='Date', y=var)
    fig.update_layout(title=f'{var} for the year 2021 in California', title_x=0.5)
    fig.show()

demand_sums = {}
solar_sums = {}
wind_sums = {}
for _, row in hourly_data.iterrows():
    hour = row['time'].split()[-1]
    demand_sums.setdefault(hour, []).append(row['demand'])
    wind_sums.setdefault(hour, []).append(row['built_wind'])
    solar_sums.setdefault(hour, []).append(row['built_solar'])
    
demand_sums = {k:sum(v)/len(v) for k, v in demand_sums.items()}
solar_sums = {k:sum(v)/len(v) for k, v in solar_sums.items()}
wind_sums = {k:sum(v)/len(v) for k, v in wind_sums.items()}



by_hour = {'Hour (PST)': [], 'Demand (MW)': [], 'Wind Supply (MW)': [], 'Solar Supply (MW)': []}
for time in demand_sums:
    by_hour['Hour (PST)'].append(datetime.strftime(datetime.strptime(time, "%H:%M:%S") - timedelta(hours=8), "%H:%M:%S"))
    by_hour['Demand (MW)'].append(demand_sums[time])
    by_hour['Wind Supply (MW)'].append(wind_sums[time])
    by_hour['Solar Supply (MW)'].append(solar_sums[time])
    
by_hour = pd.DataFrame(by_hour)
by_hour = by_hour.sort_values(by='Hour (PST)')

fig = px.line(by_hour, x='Hour (PST)', y=['Demand (MW)', 'Wind Supply (MW)', 'Solar Supply (MW)'])
fig.update_layout(width=700, height=500, title='Average Supply and Demand by Hour in California in 2021', title_x=0.5, yaxis_title = 'Supply or Demand (MW)', legend_title='')
fig.show()

235036096


## No EVs

In [56]:
total_build_factors=[1, 1.25, 1.5, 1.75, 2]
solar_build_fractions= [0, 0.25, 0.5, 0.75, 1]
ca_base_matrix, _ = scan_params(data_file='CA2021hourly.csv', total_build_factors=total_build_factors, solar_build_fractions=solar_build_fractions, num_evs=0, ev_capacity_factor=0.2, behavioral=False)
fig =  go.Figure(go.Heatmap(x= solar_build_fractions,
                            y= total_build_factors,
                            z=np.array(ca_base_matrix), 
                            colorscale='greens',
                            zmin=0.4,
                            zmax=1,
                            colorbar={'title': 'Demand Met'}))
fig.update_layout(title='', title_x=0.5, width=500, height=500, yaxis_title = 'Total Build Fraction', xaxis_title= 'Solar Fraction of Total Built')
fig.show()
fig.write_image('fig_exports/ca2023.jpeg')

In [35]:
total_build_factors=[1, 1.25, 1.5, 1.75, 2]
solar_build_fractions= [0, 0.25, 0.5, 0.75, 1]
tx_base_matrix, _ = scan_params(data_file='TX2021hourly.csv', total_build_factors=total_build_factors, solar_build_fractions=solar_build_fractions, num_evs=0, ev_capacity_factor=0.2, behavioral=False)
fig =  go.Figure(go.Heatmap(x= solar_build_fractions,
                            y= total_build_factors,
                            zmin=0.4,
                            zmax=1,
                            z=np.array(tx_base_matrix), colorscale='greens', colorbar={'title': 'Demand Met'}))
fig.update_layout(title='', title_x=0.5, width=500, height=500, yaxis_title = 'Total Build Fraction', xaxis_title= 'Solar Fraction of Total Built')
fig.show()

## 2023 EV numbers

In [81]:
total_build_factors=[1, 1.25, 1.5, 1.75, 2]
solar_build_fractions= [0, 0.25, 0.5, 0.75, 1]
ca_2023, ca_2023_frac_by_hour = scan_params(data_file='CA2021hourly.csv', total_build_factors=total_build_factors, solar_build_fractions=solar_build_fractions, num_evs=1500000, ev_capacity_factor=0.2, behavioral=False)
fig =  go.Figure(go.Heatmap(x= solar_build_fractions,
                            y= total_build_factors,
                            zmin=0.4,
                            zmax=1,
                            z=np.array(ca_2023), colorscale='greens', colorbar={'title': 'Demand Met'}))
fig.update_layout(title='', title_x=0.5, width=500, height=500, yaxis_title = 'Total Build Fraction', xaxis_title= 'Solar Fraction of Total Built')
fig.show()

fig =  go.Figure(go.Heatmap(x= solar_build_fractions,
                            y= total_build_factors,
                            zmin=0,
                            zmax=0.025,
                            z=np.array(ca_2023) - np.array(ca_base_matrix), colorscale='blues', colorbar={'title': 'Additional<br>Demand Met'}))

fig.update_layout(title='', title_x=0.5, width=500, height=500, yaxis_title = 'Total Build Fraction', xaxis_title= 'Solar Fraction of Total Built')
fig.show()

capacity_avg = {k:statistics.median(v) for k, v in ca_2023_frac_by_hour.items()}
by_hour = {'Hour (PST)': [], 'Average Storage Usage': []}
for time in capacity_avg:
    by_hour['Hour (PST)'].append(datetime.strftime(datetime.strptime(time, "%H:%M:%S") - timedelta(hours=8), "%H:%M:%S"))
    by_hour['Average Storage Usage'].append(capacity_avg[time])
fig = px.line(by_hour, 'Hour (PST)', 'Average Storage Usage')
fig.update_layout(width=700, height=500)
fig.show()


In [90]:
total_build_factors=[1, 1.25, 1.5, 1.75, 2]
solar_build_fractions= [0, 0.25, 0.5, 0.75, 1]
tx_2023, _ = scan_params(data_file='TX2021hourly.csv', total_build_factors=total_build_factors, solar_build_fractions=solar_build_fractions, num_evs=230000, ev_capacity_factor=0.2, behavioral=False)
fig =  go.Figure(go.Heatmap(x= solar_build_fractions,
                            y= total_build_factors,
                            zmin=0.4,
                            zmax=1,
                            z=np.array(tx_2023), colorscale='greens', colorbar={'title': 'Demand Met'}))
fig.update_layout(title='', title_x=0.5, width=500, height=500, yaxis_title = 'Total Build Fraction', xaxis_title= 'Solar Fraction of Total Built')
fig.show()

fig =  go.Figure(go.Heatmap(x= solar_build_fractions,
                            y= total_build_factors,
                            zmax=0.005,
                            zmin=0,
                            z=np.array(tx_2023) - np.array(tx_base_matrix), colorscale='blues', colorbar={'title': 'Additional<br>Demand Met'}))
fig.update_layout(title='', title_x=0.5, width=500, height=500, yaxis_title = 'Total Build Fraction', xaxis_title= 'Solar Fraction of Total Built')
fig.show()

## 2030 EV numbers

In [89]:
total_build_factors=[1, 1.25, 1.5, 1.75, 2]
solar_build_fractions= [0, 0.25, 0.5, 0.75, 1]
ca_2030, ca_2030_frac_by_hour = scan_params(data_file='CA2021hourly.csv', total_build_factors=total_build_factors, solar_build_fractions=solar_build_fractions, num_evs=5000000, ev_capacity_factor=0.2, behavioral=False)
fig =  go.Figure(go.Heatmap(x= solar_build_fractions,
                            y= total_build_factors,
                            zmin=0.4,
                            zmax=1,
                            z=np.array(ca_2030), colorscale='greens', colorbar={'title': 'Demand Met'}))
fig.update_layout(title='', title_x=0.5, width=500, height=500, yaxis_title = 'Total Build Fraction', xaxis_title= 'Solar Fraction of Total Built')
fig.show()

fig =  go.Figure(go.Heatmap(x= solar_build_fractions,
                            y= total_build_factors,
                            zmin=0,
                            zmax=0.1,
                            z=np.array(ca_2030) - np.array(ca_base_matrix), colorscale='blues', colorbar={'title': 'Additional<br>Demand Met'}))
fig.update_layout(title='', title_x=0.5, width=500, height=500, yaxis_title = 'Total Build Fraction', xaxis_title= 'Solar Fraction of Total Built')
fig.show()

capacity_avg = {k:statistics.median(v) for k, v in ca_2030_frac_by_hour.items()}
by_hour = {'Hour (PST)': [], 'Average Storage Usage': []}
for time in capacity_avg:
    by_hour['Hour (PST)'].append(datetime.strftime(datetime.strptime(time, "%H:%M:%S") - timedelta(hours=8), "%H:%M:%S"))
    by_hour['Average Storage Usage'].append(capacity_avg[time])
fig = px.line(by_hour, 'Hour (PST)', 'Average Storage Usage')
fig.update_layout(width=700, height=500)
fig.show()


In [92]:
total_build_factors=[1, 1.25, 1.5, 1.75, 2]
solar_build_fractions= [0, 0.25, 0.5, 0.75, 1]
tx_2030, _ = scan_params(data_file='TX2021hourly.csv', total_build_factors=total_build_factors, solar_build_fractions=solar_build_fractions, num_evs=1000000, ev_capacity_factor=0.2, behavioral=False)
fig =  go.Figure(go.Heatmap(x= solar_build_fractions,
                            y= total_build_factors,
                            zmin=0.4,
                            zmax=1,
                            z=np.array(tx_2030), colorscale='greens', colorbar={'title': 'Demand Met'}))
fig.update_layout(title='', title_x=0.5, width=500, height=500, yaxis_title = 'Total Build Fraction', xaxis_title= 'Solar Fraction of Total Built')
fig.show()

fig =  go.Figure(go.Heatmap(x= solar_build_fractions,
                            y= total_build_factors,
                            zmin=0,
                            zmax=0.025,
                            z=np.array(tx_2030) - np.array(tx_base_matrix), colorscale='blues', colorbar={'title': 'Additional<br>Demand Met'}))
fig.update_layout(title='', title_x=0.5, width=500, height=500, yaxis_title = 'Total Build Fraction', xaxis_title= 'Solar Fraction of Total Built')
fig.show()

## Behavioral Model

In [49]:
total_build_factors=[1, 1.25, 1.5, 1.75, 2]
solar_build_fractions= [0, 0.25, 0.5, 0.75, 1]
ca_2023_behavioral, _ = scan_params(data_file='CA2021hourly.csv', total_build_factors=total_build_factors, solar_build_fractions=solar_build_fractions, num_evs=1500000, ev_capacity_factor=0.2, behavioral=True)
fig =  go.Figure(go.Heatmap(x= solar_build_fractions,
                            y= total_build_factors,
                            z=np.array(ca_2023_behavioral), colorscale='matter', colorbar={'title': 'Demand Met'}))
fig.update_layout(title='', title_x=0.5, width=500, height=500, yaxis_title = 'Total Build Fraction', xaxis_title= 'Solar Fraction of Total Built')
fig.show()

fig =  go.Figure(go.Heatmap(x= solar_build_fractions,
                            y= total_build_factors,
                            zmax=0,
                            zmin=-0.002,
                            z=np.array(ca_2023_behavioral) - np.array(ca_2023), colorscale='reds_r', colorbar={'title': 'Behavioral<br>Effect'}))
fig.update_layout(title='', title_x=0.5, width=500, height=500, yaxis_title = 'Total Build Fraction', xaxis_title= 'Solar Fraction of Total Built')
fig.show()


fig =  go.Figure(go.Heatmap(x= solar_build_fractions,
                            y= total_build_factors,
                            z=np.array(ca_2023_behavioral) - np.array(ca_base_matrix), colorscale='matter', colorbar={'title': 'Additional<br>Demand Met'}))
fig.update_layout(title='', title_x=0.5, width=500, height=500, yaxis_title = 'Total Build Fraction', xaxis_title= 'Solar Fraction of Total Built')
fig.show()

In [50]:
total_build_factors=[1, 1.25, 1.5, 1.75, 2]
solar_build_fractions= [0, 0.25, 0.5, 0.75, 1]
ca_2030_behavioral, _ = scan_params(data_file='CA2021hourly.csv', total_build_factors=total_build_factors, solar_build_fractions=solar_build_fractions, num_evs=5000000, ev_capacity_factor=0.2, behavioral=True)
fig =  go.Figure(go.Heatmap(x= solar_build_fractions,
                            y= total_build_factors,
                            z=np.array(ca_2030_behavioral), colorscale='matter', colorbar={'title': 'Demand Met'}))
fig.update_layout(title='', title_x=0.5, width=500, height=500, yaxis_title = 'Total Build Fraction', xaxis_title= 'Solar Fraction of Total Built')
fig.show()

fig =  go.Figure(go.Heatmap(x= solar_build_fractions,
                            y= total_build_factors,
                            zmin = -0.002,
                            zmax = 0,
                            z=np.array(ca_2030_behavioral) - np.array(ca_2030), colorscale='reds_r', colorbar={'title': 'Behavioral<br>Effect'}))
fig.update_layout(title='', title_x=0.5, width=500, height=500, yaxis_title = 'Total Build Fraction', xaxis_title= 'Solar Fraction of Total Built')
fig.show()


fig =  go.Figure(go.Heatmap(x= solar_build_fractions,
                            y= total_build_factors,
                            z=np.array(ca_2030_behavioral) - np.array(ca_base_matrix), colorscale='matter', colorbar={'title': 'Additional<br>Demand Met'}))
fig.update_layout(title='', title_x=0.5, width=500, height=500, yaxis_title = 'Total Build Fraction', xaxis_title= 'Solar Fraction of Total Built')
fig.show()

## Second life facility

In [96]:
total_build_factors=[1, 1.25, 1.5, 1.75, 2]
solar_build_fractions= [0, 0.25, 0.5, 0.75, 1]
ca_2023_second_life, _ = scan_params(data_file='CA2021hourly.csv', total_build_factors=total_build_factors, solar_build_fractions=solar_build_fractions, num_evs=1500000, ev_capacity_factor=0.7, behavioral=True)
fig =  go.Figure(go.Heatmap(x= solar_build_fractions,
                            y= total_build_factors,
                            zmin=0.4,
                            zmax=1,
                            z=np.array(ca_2023_second_life), colorscale='greens', colorbar={'title': 'Demand Met'}))
fig.update_layout(title='', title_x=0.5, width=500, height=500, yaxis_title = 'Total Build Fraction', xaxis_title= 'Solar Fraction of Total Built')
fig.show()

fig =  go.Figure(go.Heatmap(x= solar_build_fractions,
                            y= total_build_factors,
                            zmin=0,
                            zmax=0.15,
                            z=np.array(ca_2023_second_life) - np.array(ca_base_matrix), colorscale='blues', colorbar={'title': 'Additional<br>Demand Met'}))
fig.update_layout(title='', title_x=0.5, width=500, height=500, yaxis_title = 'Total Build Fraction', xaxis_title= 'Solar Fraction of Total Built')
fig.show()

In [97]:
total_build_factors=[1, 1.25, 1.5, 1.75, 2]
solar_build_fractions= [0, 0.25, 0.5, 0.75, 1]
ca_2030_second_life, _ = scan_params(data_file='CA2021hourly.csv', total_build_factors=total_build_factors, solar_build_fractions=solar_build_fractions, num_evs=5000000, ev_capacity_factor=0.7, behavioral=True)
fig =  go.Figure(go.Heatmap(x= solar_build_fractions,
                            y= total_build_factors,
                            zmin=0.4,
                            zmax=1,
                            z=np.array(ca_2030_second_life), colorscale='greens', colorbar={'title': 'Demand Met'}))
fig.update_layout(title='', title_x=0.5, width=500, height=500, yaxis_title = 'Total Build Fraction', xaxis_title= 'Solar Fraction of Total Built')
fig.show()

fig =  go.Figure(go.Heatmap(x= solar_build_fractions,
                            y= total_build_factors,
                            zmin=0,
                            zmax=0.3,
                            z=np.array(ca_2030_second_life) - np.array(ca_base_matrix), colorscale='blues', colorbar={'title': 'Additional<br>Demand Met'}))
fig.update_layout(title='', title_x=0.5, width=500, height=500, yaxis_title = 'Total Build Fraction', xaxis_title= 'Solar Fraction of Total Built')
fig.show()